In [1]:
import pandas as pd
import json

with open("../compilation-analysis/option_columns.json","r") as f:
    option_columns = json.load(f)

#Find the dataset
df = pd.read_csv("../tuxml-datasets/dataset_encoded_size.csv", dtype={k:"int8" for k in option_columns})

df.query("cid >= 30000", inplace=True)
df.fillna(-1, inplace=True)
df.query("kernel_size >= 0", inplace=True)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import tree

def runML(hyperparams, with_tree=False):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns="cid").drop(columns="kernel_size"), df["kernel_size"], test_size=0.1, random_state=hyperparams["random_state"])  
    reg = tree.DecisionTreeRegressor(**hyperparams)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    if with_tree:
        return dfErrors["% error"].describe(), reg
    return dfErrors["% error"].describe()

In [3]:

hyperparams = {
    "criterion":"mse",
    "max_depth":12,
    "min_samples_leaf":5,
    "max_leaf_nodes":None,
    "random_state":2
}
res, reg = runML(hyperparams, with_tree=True)

In [4]:
col = df.drop(columns="cid").drop(columns="kernel_size").columns
importanceSeries = pd.Series(reg.feature_importances_, index=col.values)
importanceSeries[importanceSeries > 0].sort_values(ascending=False)

DEBUG_INFO                  3.353856e-01
RANDOMIZE_BASE              1.302672e-01
DEBUG_INFO_REDUCED          1.130614e-01
DEBUG_INFO_SPLIT            8.640952e-02
STRICT_MODULE_RWX           5.291101e-02
BLK_MQ_PCI                  3.956938e-02
MODULES                     3.568542e-02
UBSAN_SANITIZE_ALL          3.452231e-02
X86_NEED_RELOCS             1.873495e-02
KASAN                       1.080636e-02
FB_SVGALIB                  1.022115e-02
MDIO_DEVICE                 8.304272e-03
UBSAN_ALIGNMENT             7.775686e-03
KASAN_OUTLINE               6.216344e-03
GCOV_PROFILE_ALL            5.991576e-03
ARCH_MIGHT_HAVE_ACPI_PDC    5.910603e-03
SWPHY                       4.789978e-03
INTEL_GTT                   3.788891e-03
DST_CACHE                   3.737057e-03
XFS_DEBUG                   2.934081e-03
MDIO                        2.770754e-03
DRM_TTM                     2.526473e-03
SCSI_ISCSI_ATTRS            2.247725e-03
TUN_VNET_CROSS_LE           2.022638e-03
GRO_CELLS       

In [5]:
importanceSeries[importanceSeries > 0].sort_values(ascending=False).to_csv("feature_importance.csv")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
